In [1]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import svm
from sklearn.preprocessing import StandardScaler

In [2]:
# Dataset Link - https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
df = pd.read_csv("./IMDB_Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [4]:
df.drop_duplicates(inplace=True)
df.describe()

,review,sentiment
count,49582,49582
unique,49582,2
top,One of the other reviewers has mentioned that ...,positive
freq,1,24884


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49582 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     49582 non-null  object
 1   sentiment  49582 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


# Датасет

Як бачимо датасет доволі простий

Є дві колонки:

- Відгуки
- Категорія

Немає пропущених рядків, всі дані одного типу. Проте є дублікати, які ми видалили.

Також одразу можемо бачити у відгуках html код, який краще буде прибрати під час обробки тексту.

# Почнемо нашу роботу з очистки даних

Переведемо наші категорії у числа

Приберемо стоп слова

Приберемо html код

In [21]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})

df['review'] = df['review'].str.replace("<br /><br />", " ")

In [22]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming te...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [23]:
X = df['review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=23)

X_train.shape, y_train.shape

((31732,), (31732,))

In [24]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(random_state=23)),
    ])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(random_state=23))])

In [25]:
pred = text_clf.predict(X_test)
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.91      0.88      0.89      4932
           1       0.89      0.91      0.90      4985

    accuracy                           0.90      9917
   macro avg       0.90      0.90      0.90      9917
weighted avg       0.90      0.90      0.90      9917



# Результат 

З базовими параметрами модель видає результат у 90-91% на тестовому датасеті

Підберемо інші гіперпараметри та порівняємо дві моделі на валідаційному датасеті

In [26]:
upt_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(C=1.0, max_iter=50, random_state=23)),
    ])

upt_clf.fit(X_train, y_train)

upt_pred = upt_clf.predict(X_test)
print(metrics.classification_report(y_test, upt_pred))

              precision    recall  f1-score   support

           0       0.91      0.88      0.90      4932
           1       0.89      0.91      0.90      4985

    accuracy                           0.90      9917
   macro avg       0.90      0.90      0.90      9917
weighted avg       0.90      0.90      0.90      9917



# Результат

Для даного датасету стандартні гіпер параметри видають майже однаковий результат порівняно з корегованими.

Після різних підбору різних типів класифікаторів та параметрів під них, параметрів для векторизатора, результати моделі видають таку саму точність, 

проте є покраще у F-1 показнику.